In [1]:
import ipywidgets as widgets
from contextlib import ExitStack

In [ ]:
# Import the converter functions
# Putting them in a seperate notebook allows us to add
# details and descriptions without affecting the UI
%run ./converters.ipynb

# Widgets

In [2]:
# This function creates a handful of widgets that
# allow users to select the phenotype of a bird
# If we then assume all traits breed true, we
# can generate a genotype without making the
# user understand bird genetics
def createPeacockPhenotypeWidgets(peacockName, sex):
    colorList = ['Wild Type'] + list(set(
        # TECH EXPLANATION
        # lambda indicates an inline function
        # map() means that the lambda function is called for each item in the colorGenes array
        # It converts each gene's dictionary into just the name column
        # list() turns that map back into an array
        # The goal here is to make sure all genes show up as human readable options 
        # in the list, and they don't get out of sync with each other
        list(map(lambda gene: gene['name'], colorGenes)) + 
        list(map(lambda allotype: allotype['name'], sexLinkedColorAllotypes)) +
        list(map(lambda phenotype: phenotype['name'], multiGeneColors)) +
        list(map(lambda phenotype: phenotype['name'], sexAndAutosomalComboColors)) +
        # Only add the het sex colors if the bird is male - female birds can't have het sex traits
        (list(map(lambda phenotype: phenotype['name'], hetSexColors)) if sex == 'Male' else [])))
        
    
    color = widgets.Dropdown(
        options=colorList,
        value=colorList[0],
        description='Color:',
        disabled=False,
    )

    # See colorList for technical explanation
    patternList = ['Barred Wing Wild Type'] + list(set(
        list(map(lambda gene: gene['name'], patternGenes))))
    pattern = widgets.Dropdown(
        options=patternList,
        value=patternList[0],
        description='Pattern:',
        disabled=False,
    )
    
    # See colorList for technical explanation
    piedList = ['Non-Leucistic Wild Type'] + list(set(
       list(map(lambda allotype: allotype['name'], piedAllotypes)) +
       list(map(lambda phenotype: phenotype['name'], hetPied))))
    
    pied = widgets.Dropdown(
        options=piedList,
        value=piedList[0],
        description='Pied:',
        disabled=False,
    )

    eyeList = ['Non-Leucistic Wild Type'] + list(set(
       list(map(lambda allotype: allotype['name'], whiteEyeAllotypes))))
    
    eye = widgets.Dropdown(
        options=eyeList,
        value=eyeList[0],
        description='Leucistic Eye:',
        disabled=False,
    )

    return (
        widgets.VBox([
            widgets.Label(value=peacockName),
            color, pattern, pied, eye
        ]),
        {
            'color': color,
            'pattern': pattern,
            'pied': pied,
            'eye': eye
        }
    )

In [8]:
# This lets us turn a single gene into a genotype widget
# We will run this function over all the autosomal color genes
# and the shoulder genes to generate appropriate widgets
def createGenotypeWidgetFromGene(gene):
    possibilities = [
        'WT/WT', 
        'WT/' + gene['notation'],
        gene['notation'] + '/' + gene['notation']
    ]
    return widgets.Dropdown(
        options=possibilities,
        value=possibilities[0],
        description=gene['name'] + ':',
        disabled=False,
    )

# For sex-linked genes, we want to change the genotype based on sex,
# and treat different allotypes as mutually exclusive
def createMultiAllotypeGeneWidget(geneName, wildType, allotypes, sex, geneIsSexLinked = False):
    allotypesWithWild = [wildType] + allotypes
    possibilities = []

    # Iterate through each allotype as a possible first allele.
    # In this case, we use enumerate instead of a normal for each loop so we can limit
    # the loop of the second allele and prevent duplicate genotypes
    for index, firstAllele in enumerate(allotypesWithWild):
        if geneIsSexLinked and sex == 'Female':
            # For females just add the missing sex chromasome 'w' to the first allele
            possibilities.append(firstAllele['notation'] + '/w');
        else:
            # For double-allele genes, iterate through each allotype starting with the firstAllele as a possible second allele
            # This gives us every possible combination of alleles without duplicates 
            # You can think of it as the top trianlge of a possibilities square
            # The [index:] selects all items in the list starting with the same item as the current firstAllele
            for secondAllele in allotypesWithWild[index:]:
                possibilities.append(firstAllele['notation'] + '/' + secondAllele['notation'])
    
    return widgets.Dropdown(
        options=possibilities,
        value=possibilities[0],
        description=geneName+ ":",
        disabled=False,
    )

# Used for troubleshooting bugs with print()
output = widgets.Output()

# When genotype changes, update the related phenotype
# TODO - add input arrays to saveGenotypeToBird
def handleGenotypeChange(genotypeWidgets, phenotypeWidget, bird, sex, geneName, newGenotype, wildType, simpleGenes, multiAllotypeGenes, 
                         hetTraits, multiGeneTraits):
    with output:
        saveGenotypeToBird(bird, geneName, newGenotype)
        (phenotype, prevPhenotypes, unknown) = getPhenotypeFromBird(bird, sex, wildType, simpleGenes, multiAllotypeGenes, 
                                                                    hetTraits, multiGeneTraits)
        
        # Set the index and the value of the phenotype widget to match the new genotype
        # We have to set both, or else the display won't update properly
        # We also have to pause notifications on the phenotypeWidget while we update them
        # so that all the values are properly in sync
        with phenotypeWidget.hold_trait_notifications():
            phenotypeWidget.value = phenotype
            phenotypeWidget.index = phenotypeWidget.options.index(phenotype)
        

def handlePhenotypeChange(thisWidget, genotypeWidgets, bird, sex, phenotype, simpleGenes,
                          multiAllotypeGenes, sexLinkedAllotypes, multiGeneTraits, sexAndAutosomalCombos, hetSexTraits):
    with output:
        savePhenotypeToBird(bird, phenotype, sex, simpleGenes, multiAllotypeGenes, sexLinkedAllotypes, 
                            multiGeneTraits, sexAndAutosomalCombos, hetSexTraits)

        # We need to wait until all genotype widgets have been updated before handling the update notification
        # so that multi-gene traits don't erase themselves
        # There is really not an easily non-technical way to translate the next two lines, which holds the notifications
        # basically, it tells each genotype widget to wait to send notifications, and then creates a code block that will
        # run while those notifications are being paused
        # Once the code block is finished, then all notifications will be unpaused and fired at once
        with ExitStack() as stack:
            holds = [stack.enter_context(w.hold_trait_notifications()) for w in colorArray]
            
            for widget in genotypeWidgets:
                if widget.description[:-1] in bird:
                    widget.value = bird[widget.description[:-1]]
                    widget.index = widget.options.index(bird[widget.description[:-1]])
                else:
                    widget.value = widget.options[0]
                    widget.index = 0

def createPeacockGenotypeWidgets(peacockName, sex, phenotypeWidgets):
    bird = {}
    
    # For each possible color, create a widget with genotype options
    colors = []
    for gene in colorGenes:
        widget = createGenotypeWidgetFromGene(gene)
        colors.append(widget)
        # This updates the color phenotype whenever any color genotype changes
        widget.observe(
            lambda changedValue, geneName=gene['name']: 
                handleGenotypeChange(colors, phenotypeWidgets['color'], bird, sex, geneName, changedValue['new'], 
                                     "Wild Type", colorGenes, 
                                     [{ 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }],
                                     hetSexColors, multiGeneColors), 
            'value')

    # Create a single widget for all the sex-linked color varients
    sexLinkedColor = createMultiAllotypeGeneWidget('Sex-Linked Color', { 'notation': 'Z(WT)', 'name': 'Wild Type' }, 
                                                   sexLinkedColorAllotypes, sex, True)
    colors.append(sexLinkedColor)
    # This updates the phenotype whenever the sex-linked color genotype changes
    sexLinkedColor.observe(
        lambda changedValue, sex=sex: 
            handleGenotypeChange(colors, phenotypeWidgets['color'], bird, sex, 'Sex-Linked Color', changedValue['new'],
                                 "Wild Type", colorGenes, 
                                 [{ 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }],
                                 hetSexColors, multiGeneColors), 
        'value')

    # This updates the genotype(s) when the color phenotype changes
    phenotypeWidgets['color'].observe(
        lambda changedValue: 
            handlePhenotypeChange(phenotypeWidgets['color'], colors, bird, sex, changedValue['new'], 
                                  colorGenes, 
                                  [{ 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }], 
                                  sexLinkedColorAllotypes, multiGeneColors,  sexAndAutosomalComboColors, hetSexColors), 
            'value')

    # Create a single widget for Pied genes
    pied = createMultiAllotypeGeneWidget('Pied', { 'notation': 'WT', 'name': 'Non-Leucistic Wild Type' }, 
                                                   piedAllotypes, sex)
    # This updates the phenotype whenever the sex-linked color genotype changes
    pied.observe(
        lambda changedValue, sex=sex: 
            handleGenotypeChange([pied], phenotypeWidgets['pied'], bird, sex, 'Pied', changedValue['new'], 
                                "Non-Leucistic Wild Type", [], 
                                [{ 'name': 'Pied', 'allotypes': piedAllotypes, 'sexLinked': False }],
                                hetPied, []), 
        'value')

    # This updates the genotype(s) when the color phenotype changes
    phenotypeWidgets['pied'].observe(
        lambda changedValue: 
            handlePhenotypeChange(phenotypeWidgets['pied'], [pied], bird, sex, changedValue['new'], 
                                       [], 
                                       [{ 'name': 'Pied', 'allotypes': piedAllotypes, 'sexLinked': False }], 
                                       [], [],  [], hetPied), 
            'value')

    # Create a single widget for Leucistic Eye genes
    whiteEye = createMultiAllotypeGeneWidget('Leucistic Eye', { 'notation': 'WT', 'name': 'Non-Leucistic Wild Type' }, 
                                                   whiteEyeAllotypes, sex)
    # This updates the phenotype whenever the Leucistic Eye genotype changes
    whiteEye.observe(
        lambda changedValue, sex=sex: 
            handleGenotypeChange([whiteEye], phenotypeWidgets['eye'], bird, sex, 'Leucistic Eye', changedValue['new'], 
                                "Non-Leucistic Wild Type", [], 
                                [{ 'name': 'Leucistic Eye', 'allotypes': whiteEyeAllotypes, 'sexLinked': False }],
                                hetWhite, []),'value')

    # This updates the genotype(s) when the Leucistic Eye phenotype changes
    phenotypeWidgets['eye'].observe(
        lambda changedValue: 
            handlePhenotypeChange(phenotypeWidgets['eye'], [whiteEye], bird, sex, changedValue['new'], [], 
                                  [{ 'name': 'Leucistic Eye', 'allotypes': whiteEyeAllotypes, 'sexLinked': False }], 
                                  [], [], [], hetWhite), 
        'value')

    return (
        widgets.VBox([widgets.Label(value=peacockName)] + colors + [pied, whiteEye]),
        {
            'colors': colors,
            'pied': pied,
            'eye': whiteEye
        }
    )

In [ ]:
def generatePeafowlImage(sex, color):
    # Color should be the base layer
    colorImageName = "Images/" + sex + "/Color/" + color + ".png"
    if not os.path.isfile(colorImageName):
        # Default to the unknown color if we don't have an image
        # for the defined color
        colorImageName = "Images/" + sex + "/Color/Unknown.png"

    # Open the file for the color image
    peafowlColor = Image.open(colorImageName)
    # Open the lineart image
    peafowlLineart = Image.open("Images/" + sex + "/lineart.png")

    # Merge the images as layers
    peafowlColor.paste(peafowlLineart, (0, 0), peafowlLineart)

    # Extract the image data as bytes
    imageData = io.BytesIO()
    peafowlColor.save(imageData, format='PNG')

    # Return a widget-compatible image
    return imageData

In [ ]:
# Define color genes
colorGenes = [
    # Note: the "default" gene aka Wild Type is not included because it is a special
    # case that can apply to alleles of any gene
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'o',       'name': 'Opal' },
    { 'notation': 'md',      'name': 'Midnight' },
    { 'notation': 'j',       'name': 'Jade' },
    { 'notation': 'mo',      'name': 'Montana' },
    { 'notation': 'ch',      'name': 'Charcoal' },
    { 'notation': 'st',      'name': 'Steel' },
    { 'notation': 'um',      'name': 'Ultramarine' },
    { 'notation': 'bu',      'name': 'Burnt Umber' }
]

sexLinkedColorAllotypes = [
    { 'notation': 'Z(c)',    'name': 'Cameo' },
    { 'notation': 'Z(pl)',   'name': 'American Purple' },
    { 'notation': 'Z(va)',   'name': 'Sonja\'s Violet' },
    { 'notation': 'Z(ve)',   'name': 'European Violet' },
    # Note: even though peach is actually 2 genes, we are treating
    # it as one for the sake of this code, and dealing with het Peach
    # as a special phenotype
    { 'notation': 'Z(pl:c)', 'name': 'Peach' }
]

# Define what genes combinations form special colors
multiGeneColors = [
    { 'name': 'Platinum',   'genesNeeded': ['Bronze', 'Opal' ] },
    { 'name': 'Taupe',      'genesNeeded': ['American Purple', 'Opal' ] },
    { 'name': 'Mocha',      'genesNeeded': ['American Purple', 'Midnight' ] },
    { 'name': 'Ivory',      'genesNeeded': ['Cameo', 'Opal' ] },
    { 'name': 'Indigo',     'genesNeeded': ['American Purple', 'Bronze' ] },
    { 'name': 'Hazel',      'genesNeeded': ['American Purple', 'Bronze' ] },
    { 'name': 'Cinnamon',   'genesNeeded': ['Cameo', 'Bronze' ] }
]

sexAndAutosomalComboColors = [
    { 'name': 'Taupe',         'autosomalGene': 'o',  'sexGene': 'Z(pl)' },
    { 'name': 'Mocha',         'autosomalGene': 'md', 'sexGene': 'Z(pl)' },
    { 'name': 'Ivory',         'autosomalGene': 'o',  'sexGene': 'Z(c)' },
    { 'name': 'Indigo',        'autosomalGene': 'br', 'sexGene': 'Z(pl)' },
    { 'name': 'Hazel',         'autosomalGene': 'br', 'sexGene': 'Z(pl)' }
]

hetSexColors = [
    { 'name': 'Midway between Violet and Purple', 'geneName': 'Sex-Linked Color', 'alleles': ['Z(pl)', 'Z(ve)'] },
    { 'name': 'Cameo', 'geneName': 'Sex-Linked Color', 'alleles': ['Z(c)', 'Z(pl:c)'] },
    { 'name': 'American Purple', 'geneName': 'Sex-Linked Color', 'alleles': ['Z(pl)', 'Z(pl:c)'] }
]

# Define pattern genes
patternGenes = [
    { 'notation': 'bs',      'name': 'Blackshoulder' }
]

# Define leucistic genes
leucisticGenes = [
    { 'notation': 'p',       'name': 'Pied' },
    { 'notation': 'WE',       'name': 'White Eye' }
]

piedAllotypes = [
    { 'notation': 'p',       'name': 'Dark Pied' },
    { 'notation': 'W',       'name': 'White' }
]

# Pied special cases
hetPied = [
    { 'name': 'Pied',      'geneName': 'Pied', 'alleles': ['p', 'W'] },
    { 'name': 'Dark Pied', 'geneName': 'Pied', 'alleles': ['WT', 'W'] },
    { 'name': 'Dark Pied', 'geneName': 'Pied', 'alleles': ['WT', 'p'] }
]

whiteEyeAllotypes = [
    { 'notation': 'WE',      'name': 'White Eye' },
    { 'notation': 'sWE',     'name': 'Silver White Eye' }
]

hetWhite = [
    { 'name': 'Silver White Eye',      'geneName': 'White Eye', 'alleles': ['WE', 'sWE'] },
    { 'name': 'Dark Pied', 'geneName': 'Pied', 'alleles': ['WT', 'W'] },
    { 'name': 'Dark Pied', 'geneName': 'Pied', 'alleles': ['WT', 'p'] }
]

In [ ]:
colorList = ['Wild Type'] + list(set(
        # TECH EXPLANATION
        # lambda indicates an inline function
        # map() means that the lambda function is called for each item in the colorGenes array
        # It converts each gene's dictionary into just the name column
        # list() turns that map back into an array
        # The goal here is to make sure all genes show up as human readable options 
        # in the list, and they don't get out of sync with each other
        list(map(lambda gene: gene['name'], colorGenes)) + 
        list(map(lambda allotype: allotype['name'], sexLinkedColorAllotypes)) +
        list(map(lambda phenotype: phenotype['name'], multiGeneColors)) +
        list(map(lambda phenotype: phenotype['name'], sexAndAutosomalComboColors)) +
        # Only add the het sex colors if the bird is male - female birds can't have het sex traits
        (list(map(lambda phenotype: phenotype['name'], hetSexColors)))))

color = widgets.Dropdown(
    options=colorList,
    value=colorList[0],
    description='Color:',
    disabled=False,
)

colors = {}
colorArray = []
for gene in colorGenes:
    widget = createGenotypeWidgetFromGene(gene)
    colors[gene['name']] = widget
    colorArray.append(widget)

def updateBird(bird, phenotype, **kwargs):
    output.clear_output()
    with output:
        (currPhenotype, phenotypes, unknown) = getPhenotypeFromBird(bird, 'Male', 'Wild Type', colorGenes, 
                                               [{ 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }], 
                                               hetSexColors, multiGeneColors)
        if phenotype != currPhenotype:
            savePhenotypeToBird(bird, phenotype, 'Male', colorGenes, 
                                [{ 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }], 
                                sexLinkedColorAllotypes, multiGeneColors,  sexAndAutosomalComboColors, hetSexColors)
            
            # We need to wait until all genotype widgets have been updated before handling the update notification
            # so that multi-gene traits don't erase themselves
            # There is really not an easily non-technical way to translate the next two lines, which holds the notifications
            # basically, it tells each genotype widget to wait to send notifications, and then creates a code block that will
            # run while those notifications are being paused
            # Once the code block is finished, then all notifications will be unpaused and fired at once
            with ExitStack() as stack:
                holds = [stack.enter_context(w.hold_trait_notifications()) for w in colorArray]
                
                for widget in colorArray:
                    with widget.hold_trait_notifications():
                        if widget.description[:-1] in bird:
                            widget.value = bird[widget.description[:-1]]
                            widget.index = widget.options.index(bird[widget.description[:-1]])
                        else:
                            widget.value = widget.options[0]
                            widget.index = 0
        else:
            for gene in colorGenes:
                saveGenotypeToBird(bird, gene['name'], kwargs[gene['name']])

            (newPhenotype, phenotypes, unknown) = getPhenotypeFromBird(bird, 'Male', 'Wild Type', colorGenes, 
                                                  [{ 'name': 'Sex-Linked Color', 'allotypes': sexLinkedColorAllotypes, 'sexLinked': True }], 
                                                  hetSexColors, multiGeneColors)
            
            with color.hold_trait_notifications():
                color.index = color.options.index(newPhenotype)
                color.value = newPhenotype
            
# updateBirdDict = { 'bird': widgets.fixed({}), 'phenotype':color, }
# updateBirdDict.update(colors)
# widgets.interactive_output(updateBird, updateBirdDict);
# display(widgets.VBox([color, widgets.VBox(colorArray), output]))